In [ ]:
!pip install pymysql sqlalchemy

In [ ]:
!pip install cryptography

In [8]:
from sqlalchemy import create_engine
import pandas as pd
username="root"
password="root"
host="localhost"
port="3306"
database="coustmer_bhv"

engine= create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")


--Q1. What is the total revenue generated by male vs. female customers?

--Q2. Which customers used a discount but still spent more than the average purchase amount? 

-- Q3. Which are the top 5 products with the highest average review rating?

--Q4. Compare the average Purchase Amounts between Standard and Express Shipping. 

--Q5. Do subscribed customers spend more? Compare average spend and total revenue between subscribers and non-subscribers.

--Q6. Which 5 products have the highest percentage of purchases with discounts applied?

--Q7. Segment customers into New, Returning, and Loyal based on their total number of previous purchases, and show the count of each segment. 

--Q8. What are the top 3 most purchased products within each category? 

--Q9. Are customers who are repeat buyers (more than 5 previous purchases) also likely to subscribe?

--Q10. What is the revenue contribution of each age group? 

In [9]:
#--Q1. What is the total revenue generated by male vs. female customers?
text="""
SELECT
gender,SUM(purchase_amount) AS total_revenue
FROM mytable
GROUP BY gender;"""
pd.read_sql(text,engine)

,gender,total_revenue
0,Male,157890.0
1,Female,75191.0


In [10]:
#--Q2. Which customers used a discount but still spent more than the average purchase amount? 
text="""
SELECT
customer_id,purchase_amount
FROM mytable
WHERE discount_applied = 'Yes' and purchase_amount >= (SELECT AVG(purchase_amount) FROM mytable);"""
pd.read_sql(text,engine)

,customer_id,purchase_amount
0,2,64
1,3,73
2,4,90
3,7,85
4,9,97
...,...,...
834,1667,64
835,1671,73
836,1673,73
837,1674,62


In [ ]:
#-- Q3. Which are the top 5 products with the highest average review rating
text="""
SELECT
item_purchased,AVG(review_rating) AS avg_rating 
FROM mytable
GROUP BY item_purchased ORDER BY avg_rating DESC LIMIT 5;"""
pd.read_sql(text,engine)

,item_purchased,avg_rating
0,Gloves,3.861429
1,Sandals,3.844375
2,Boots,3.818750
3,Hat,3.801299
4,Skirt,3.784810


In [ ]:
#--Q4. Compare the average Purchase Amounts between Standard and Express Shipping. 
text="""
select shipping_type, 
ROUND(AVG(purchase_amount),2)
from mytable
where shipping_type in ('Standard','Express')
group by shipping_type;"""
pd.read_sql(text,engine)

,shipping_type,"ROUND(AVG(purchase_amount),2)"
0,Express,60.48
1,Standard,58.46


In [ ]:
#Q5. Do subscribed customers spend more? Compare average spend and total revenue 
#between subscribers and non-subscribers.
text="""
SELECT subscription_status,
       COUNT(customer_id) AS total_customers,
       ROUND(AVG(purchase_amount),2) AS avg_spend,
       ROUND(SUM(purchase_amount),2) AS total_revenue
FROM mytable
GROUP BY subscription_status
ORDER BY total_revenue,avg_spend DESC;"""
pd.read_sql(text,engine)


,subscription_status,total_customers,avg_spend,total_revenue
0,Yes,1053,59.49,62645.0
1,No,2847,59.87,170436.0


In [ ]:
#Q6. Which 5 products have the highest percentage of purchases with discounts applied?

text="""
SELECT item_purchased,
       ROUND(100.0 * SUM(CASE WHEN discount_applied = 'Yes' THEN 1 ELSE 0 END)/COUNT(*),2) AS discount_rate
FROM mytable
GROUP BY item_purchased
ORDER BY discount_rate DESC
LIMIT 5;"""
pd.read_sql(text,engine)

In [ ]:
#Q7. Segment customers into New, Returning, and Loyal based on their total 
#number of previous purchases, and show the count of each segment. 

text="""
WITH customer_cat AS(
SELECT customer_id,previous_purchases,
CASE 
    WHEN previous_purchases = 1 THEN 'New' 
    WHEN previous_purchases BETWEEN 2 AND 10 THEN 'Returning'
    ELSE 'Loyal'
    END AS customer_segment
FROM mytable)

SELECT customer_segment,COUNT(*) AS 'No Of Customer'
FROM customer_cat 
GROUP BY customer_segment;"""
pd.read_sql(text,engine)

In [ ]:
#Q8. What are the top 3 most purchased products within each category? 
text="""
WITH item_counts AS (
    SELECT category,
           item_purchased,
           COUNT(customer_id) AS total_orders,
           ROW_NUMBER() OVER (PARTITION BY category ORDER BY COUNT(customer_id) DESC) AS item_rank
    FROM mytable
    GROUP BY category, item_purchased
)
SELECT item_rank,category, item_purchased, total_orders
FROM item_counts
WHERE item_rank <=3;"""
pd.read_sql(text,engine)

In [ ]:
#Q9. Are customers who are repeat buyers (more than 5 previous purchases) also likely to subscribe?
text="""
SELECT subscription_status,
       COUNT(customer_id) AS repeat_buyers
FROM mytable
WHERE previous_purchases > 5
GROUP BY subscription_status;"""
pd.read_sql(text,engine)


In [ ]:
#Q10. What is the revenue contribution of each age group? 
text="""
SELECT age_group,
       SUM(purchase_amount) AS amount
FROM mytable
GROUP BY age_group
ORDER BY amount desc;"""
pd.read_sql(text,engine)